# Integrating a user-provided simulator in an end-to-end AutoEmulate workflow 

## Overview

<b>In this workflow we demonstrate the integration of a Cardiovascular simulator, Naghavi Model from ModularCirc in the end-to-end AutoEmulate workflow.</b> 

Naghavi model is a 0D (zero-dimensional) computational model of the cardiovascular system, which is used to simulate blood flow and pressure dynamics in the heart and blood vessels.

This demo includes:
- Setting up parameter ranges 
- Creating samples
- Running the simulator to generate training data for the emulator 
- Using AutoEmulate to find the best pre-processing technique and model tailored to the simulation data 
- Applying history matching to refine the model and enhance parameter ranges 
- Sensitivity Analysis 


<img src="https://raw.githubusercontent.com/alan-turing-institute/autoemulate/refs/heads/main/misc/workflow.png" alt="Work Flow" style="width:100%;"/>


## Additional dependency requirements

<b>In this demonstration we are using the Naghavi Model Simulator from ModularCirc library. Therefore, the user needs to install the ModularCirc library in their existing AutoEmulate virtual environemnt as an additional dependency.</b> 

In [1]:
#! pip install git+https://github.com/alan-turing-institute/ModularCirc.git@dev

## Workflow

#### 1 - Create a dictionary called `parameters_range` which contains the name of the simulator input parameters and their range. In this case, we have an imported function to do this. 

The `parameter_range` dictionary is a string to tuple mapping where the string is the name of the parameter and the tuple is the range of the parameter. The range is defined as a tuple of two values, the minimum and maximum value of the parameter. 



In [2]:
from autoemulate.simulations.naghavi_cardiac_ModularCirc import extract_parameter_ranges
# Usage example:
parameters_range = extract_parameter_ranges('../data/naghavi_model_parameters.json')
parameters_range

/home/edwin/anaconda3/envs/autoemulate/lib/python3.12/site-packages/pandera/engines/pandas_engine.py:67: UserWarning: Using typeguard < 3. Generic types like List[TYPE], Dict[TYPE, TYPE] will only validate the first element in the collection.
  warnings.warn(


{'ao.r': (120.0, 360.0),
 'ao.c': (0.15, 0.44999999999999996),
 'art.r': (562.5, 1687.5),
 'art.c': (1.5, 4.5),
 'ven.r': (4.5, 13.5),
 'ven.c': (66.65, 199.95000000000002),
 'av.r': (3.0, 9.0),
 'mv.r': (2.05, 6.1499999999999995),
 'la.E_pas': (0.22, 0.66),
 'la.E_act': (0.225, 0.675),
 'la.v_ref': (5.0, 15.0),
 'la.k_pas': (0.01665, 0.07500000000000001),
 'lv.E_pas': (0.5, 1.5),
 'lv.E_act': (1.5, 4.5),
 'lv.v_ref': (5.0, 15.0),
 'lv.k_pas': (0.00999, 0.045)}

#### 2 - Wrap your Simulator in the AutoEmulate Simulator Base Class.

Autoemulate has a Simulator class that all simulators should inherit from. This class has one abstract method that the user must implement called _forward. This method is responsible for accepting some input parameters and outputting the results from a single simulation results. Both the input and output of this method should be an 2D array/Tensor where the dimensions are (n_samples, n_features).


Please refer to the [Custom Simulators](https://autoemulate.readthedocs.io/en/latest/tutorials/08_Custom_Simulations.html) tutorial for more information on how to wrap your simulator in the AutoEmulate Simulator Base Class. 


In [3]:
from re import X
from sys import exception
from autoemulate.experimental.simulations.base import Simulator

import numpy as np
import pandas as pd
import torch
from ModularCirc.Models.NaghaviModel import NaghaviModel
from ModularCirc.Models.NaghaviModel import NaghaviModelParameters
from ModularCirc.Solver import Solver

class NaghaviSimulator(Simulator):
    def __init__(
        self,
        parameters_range,
        output_names,
        n_cycles: int = 40,
        dt: float = 0.001,
    ):
        """
        Initialize the Naghavi simulator with specific parameters.
        Some default parameter ranges can be found
        autoemulate.simulations.naghavi_cardiac_ModularCirc.py

        Parameters
        ----------
        parameters_range : dict
            Dictionary mapping parameter names to (min, max) tuples.
        output_names : list
            List of specific output names to track.
        n_cycles : int
            Number of simulation cycles.
        dt : float
            Time step size.
        """
        super().__init__(parameters_range, output_names)

        # Naghavi-specific attributes
        self.n_cycles = n_cycles
        self.dt = dt
        self.time_setup = {
            "name": "HistoryMatching",
            "ncycles": n_cycles,
            "tcycle": 1.0,
            "dt": dt,
            "export_min": 1,
        }

    def _forward(self, x):
        """
        Run a single Naghavi model simulation and return output statistics.

        Args:
            x: TensorLike
                Input parameters for the simulation

        Returns:
            Array of output statistics or None if simulation fails
        """
        if x.shape[-1] != len(self._param_names):
            raise ValueError(
                f"Input x must have the same shape as the number of parameters:"
                f" {len(self._param_names)}"
            )

        # Drop first dim
        x = x.squeeze(0)

        # Set parameter object
        parobj = NaghaviModelParameters()

        for i, param_name in enumerate(self._param_names):
            if param_name == "T":
                continue

            obj, param = param_name.split(".")
            value = x[i].numpy()
            parobj._set_comp(obj, [obj], **{param: value})

        # Set cycle time
        t_cycle = (
            x[self.get_parameter_idx("T")].item() if "T" in self._param_names else 1.0
        )

        self.time_setup["tcycle"] = t_cycle

        # Run simulation
        model = NaghaviModel(
            time_setup_dict=self.time_setup, parobj=parobj, suppress_printing=True
        )
        solver = Solver(model=model)
        solver.setup(suppress_output=True, optimize_secondary_sv=False, method="LSODA")
        solver.solve()

        if not solver.converged:
            err_msg = "Solver did not converge"
            raise Exception(err_msg)

        # Collect and process outputs
        output_stats = []
        output_names = []

        for component_name, component_obj in model.components.items():
            for attr_name in dir(component_obj):
                if (
                    not attr_name.startswith("_")
                    and attr_name != "kwargs"
                    and not callable(getattr(component_obj, attr_name))
                ):
                    try:
                        attr = getattr(component_obj, attr_name)
                        if hasattr(attr, "values"):
                            full_name = f"{component_name}.{attr_name}"

                            # Check if we should track this variable
                            if (
                                not self.output_names
                                or full_name in self.output_names
                            ):
                                values = np.array(attr.values)

                                # Use the base class method to calculate statistics
                                stats, stat_names = self._calculate_output_stats(
                                    values, full_name
                                )
                                output_stats.extend(stats)
                                output_names.extend(stat_names)
                    except Exception as e:
                        print(e)
                        continue

        # Always update output names after the first simulation
        if not self._has_sample_forward:
            #self._output_names = output_names
            self._has_sample_forward = True

        # Convert output stats to a tensor
        return torch.Tensor(output_stats).unsqueeze(0)

    def _calculate_output_stats(
        self, output_values: np.ndarray, base_name: str
    ) -> tuple[np.ndarray, list[str]]:
        """
        Calculate statistics for an output time series.

        Args:
            output_values: Array of time series values
            base_name: Base name of the output variable

        Returns:
            Tuple of (stats_array, stat_names)
        """
        stats = np.array(
            [
                np.min(output_values),
                np.max(output_values),
                np.mean(output_values),
                np.max(output_values) - np.min(output_values),
            ]
        )

        stat_names = [
            f"{base_name}_min",
            f"{base_name}_max",
            f"{base_name}_mean",
            f"{base_name}_range",
        ]

        return stats, stat_names

In [4]:
# Initialize simulator with specific outputs
simulator = NaghaviSimulator(
    parameters_range=parameters_range,
    output_names=['lv.P_i', 'lv.P_o'],  # Only the ones you're interested in
    n_cycles=300,
    dt=0.001,
)

#### 3. Generate input samples from given parameter ranges.

The Simulator class has a built-in method for generating input samples from the given parameter ranges using Latin Hypercube Sampling.

In [6]:
input_samples = simulator.sample_inputs(
    n_samples=20,
)
input_samples.shape

torch.Size([20, 16])

We can see the shape of the input samples is (n_samples, n_parameters).

There is a useful function called get_parameter_idx that will return the parameter index for a specific parameter name. For example, the code below shows how to extract from the input samples the value of the parameter la.E_pas


In [7]:
param_idx = simulator.get_parameter_idx('la.E_pas')
input_samples[:, param_idx]

tensor([0.3639, 0.3370, 0.5370, 0.4422, 0.4683, 0.2758, 0.5916, 0.4131, 0.3194,
        0.2340, 0.5527, 0.2429, 0.3071, 0.6115, 0.3901, 0.4198, 0.5215, 0.6526,
        0.4869, 0.6189])

#### 4 - The simulator class can now be run either by a single time using the forward method or by using run_batch_simulations to obtain data for training AutoEmulate.

In [8]:
# Run a single simulation - NOTE the input sample must be a 2D array.
single_output = simulator.forward(input_samples[0:1, :])
print(f"Single forward simulation output shape: {single_output.shape}")

batch_output = simulator.forward_batch(input_samples)
print(f"Batch forward simulation output shape: {batch_output.shape}")

Single forward simulation output shape: torch.Size([1, 8])


Running simulations: 100%|██████████| 20/20 [00:38<00:00,  1.95s/it]

Successfully completed 20/20 simulations (100.0%)
Batch forward simulation output shape: torch.Size([20, 8])


In [9]:
input_samples.shape

torch.Size([20, 16])

In [10]:
batch_output.shape

torch.Size([20, 8])

#### 5 - Run AutoEmulate.

We will now run initialise the main AutoEmulate class. To initialise the class, we need to pass the input and output samples and the type of models we would like to train as emulators.

In [11]:
import numpy as np
from autoemulate.experimental.compare import AutoEmulate
from autoemulate.experimental.emulators import ALL_EMULATORS

em = AutoEmulate(x=input_samples, y=batch_output, models=[ALL_EMULATORS[0], ALL_EMULATORS[4], ALL_EMULATORS[5]])


#### 6 - Run compare to train AutoEmulate and extract the best model.

We can now run the compare method to train AutoEmulate and extract the best model. The compare method will return the best model based on the given input and output samples. Under the hood, the following steps are performed for each model type:

1. Tune the model hyperparameters using the input and output samples.
2. Train the model using the input and output samples with cross-validation to get a score for each model.
3. Returns a dictionary with all models and their scores.

In [13]:
import logging
logging.basicConfig(level=logging.INFO)

In [14]:
model_scores = em.compare(3)

INFO:autoemulate.experimental.compare:Training model: GaussianProcessExact
  0%|          | 0/3 [00:00<?, ?it/s]INFO:autoemulate.experimental.emulators.gaussian_process.exact:Epoch:   1/200; MLL: -40.201; noise: 0.693
INFO:autoemulate.experimental.emulators.gaussian_process.exact:Epoch:   2/200; MLL: -39.482; noise: 0.702
INFO:autoemulate.experimental.emulators.gaussian_process.exact:Epoch:   3/200; MLL: -38.781; noise: 0.712
INFO:autoemulate.experimental.emulators.gaussian_process.exact:Epoch:   4/200; MLL: -38.095; noise: 0.721
INFO:autoemulate.experimental.emulators.gaussian_process.exact:Epoch:   5/200; MLL: -37.427; noise: 0.731
INFO:autoemulate.experimental.emulators.gaussian_process.exact:Epoch:   6/200; MLL: -36.775; noise: 0.740
INFO:autoemulate.experimental.emulators.gaussian_process.exact:Epoch:   7/200; MLL: -36.139; noise: 0.750
INFO:autoemulate.experimental.emulators.gaussian_process.exact:Epoch:   8/200; MLL: -35.520; noise: 0.760
INFO:autoemulate.experimental.emulators.

In [ ]:
model_scores

#### 7 - Examine the summary of cross-validation.

In [ ]:
for model_name, model in model_scores.items():
    print(f"Model: {model_name}")
    print(f"rmse_score: {model['rmse_score']}")
    print(f"r2_score: {model['r2_score']}")


In [ ]:
em.models[0]

#### 9 - Sensitivity Analysis 
Use AutoEmulate to perform sensitivity analysis. This will help identify the parameters that have higher impact on the outputs to narrow down the search space for performing model calibration. 

Sobol Interpretation:

- $S_1$ values sum to ≤ 1.0 (exact fraction of variance explained)
- $S_t - S_1$ = interaction effects involving that parameter
- Large $S_t - S_1$ gap indicates strong interactions

Morris Interpretation:

- High $\mu^*$, Low $\sigma$: Important parameter with linear/monotonic effects
- High $\mu^*$, High $\sigma$: Important parameter with non-linear effects or interactions
- Low $\mu^*$, High $\sigma$: Parameter involved in interactions but not individually important
- Low $\mu^*$, Low $\sigma$: Unimportant parameter

In [ ]:
# Extract parameter names and bounds from the dictionary
parameter_names = list(parameters_range.keys())
parameter_bounds = list(parameters_range.values())

# Define the problem dictionary for Sobol sensitivity analysis
problem = {
    'num_vars': len(parameter_names),
    'names': parameter_names,
    'bounds': parameter_bounds
}


In [ ]:
si = em.sensitivity_analysis(problem=problem, method='sobol')
si.head()

In [ ]:
em.plot_sensitivity_analysis(si)


### Refining the Model with Real-World Observations

To refine our emulator, we need real-world observations to compare against. These observations can come from:
- Experimental values from literature
- Simulation results from a known reliable parameter set

In this example, we'll generate our observations by running the simulator at the midpoint of each parameter range, treating these as our "ground truth" values for calibration. Note that in a real world example one can have multiple observations.

In [ ]:

# An example of how to define observed data with means and variances from a hypothetical experiment
observations = {
    'lv.P_i_min': (5.0, 0.1),   # Minimum of minimum LV pressure
    'lv.P_i_max': (20.0, 0.1),   # Maximum of minimum LV pressure
    'lv.P_i_mean': (10.0, 0.1),  # Mean of minimum LV pressure
    'lv.P_i_range': (15.0, 0.5), # Range of minimum LV pressure
    'lv.P_o_min': (1.0, 0.1),  # Minimum of maximum LV pressure
    'lv.P_o_max': (13.0, 0.1),  # Maximum of maximum LV pressure
    'lv.P_o_mean': (12.0, 0.1), # Mean of maximum LV pressure
    'lv.P_o_range': (20.0, 0.5)  # Range of maximum LV pressure
}

In [ ]:
# Otherwise, use one forward pass of your simualtion to get the observed data
# Calculate midpoint parameters
midpoint_params = {}
for param_name, (min_val, max_val) in parameters_range.items():
    midpoint_params[param_name] = (min_val + max_val) / 2.0
# Run the simulator with midpoint parameters
midpoint_results = simulator.sample_forward(midpoint_params)

In [ ]:
# Create observations dictionary
observations = {}
output_names = simulator.output_names
observations = {name: (float(val), max(abs(val) * 0.01, 0.01)) for name, val in zip(output_names, midpoint_results)}
observations


#### 10 - History Matching
 
Once you have the final model, running history matching can improve your model. The Implausibility metric is calculated using the following relation for each set of parameter:

$I_i(\overline{x_0}) = \frac{|z_i - \mathbb{E}(f_i(\overline{x_0}))|}{\sqrt{\text{Var}[z_i - \mathbb{E}(f_i(\overline{x_0}))]}}$
Where if implosibility ($I_i$) exceeds a threshhold value, the points will be rulled out. 
The outcome of history matching are the NORY (Not Ruled Out Yet) and RO (Ruled Out) points.

- create a dictionary of your observations, this should match the output names of your simulator 
- create the history matching object 
- run history matching 


In [ ]:
from autoemulate.history_matching import HistoryMatching

# Create history matcher
hm = HistoryMatching(
    simulator=simulator,
    observations=observations,
    threshold=1.0
)

# Run history matching
all_samples, all_impl_scores, emulator = hm.run(
    n_waves=50,
    n_samples_per_wave=100,
    emulator_predict=True,
    initial_emulator=gp_final,
)

In [ ]:

# Simple NROY extraction - just check the threshold!
threshold = 1.0  # Same threshold used in history matching

# Find samples where ALL outputs have implausibility <= threshold
nroy_mask = np.all(all_impl_scores <= threshold, axis=1)
nroy_indices = np.where(nroy_mask)[0]
nroy_samples = all_samples[nroy_indices]

print(f"Total samples: {len(all_samples)}")
print(f"NROY samples: {len(nroy_samples)}")
print(f"NROY percentage: {len(nroy_samples)/len(all_samples)*100:.1f}%")

In [ ]:
from autoemulate.history_matching_dashboard import HistoryMatchingDashboard
dashboard = HistoryMatchingDashboard(
    samples=all_samples,
    impl_scores=all_impl_scores,
    param_names=simulator.param_names,
    output_names=simulator.output_names,
    )
dashboard.display()

<img src="https://raw.githubusercontent.com/alan-turing-institute/autoemulate/refs/heads/main/misc/vis_dashboard_pic_sample.png" alt="Work Flow" style="width:20%;"/> """

#### 11 - MCMC
Once you have identified the important parameters through the Sensitivity analysis tool, the MCMC module can return the calibrated parameter values with uncertainty. 
The MCMC algorithm tries to find parameter values that match the predictions by the emulator to your `observations` whilst staying within the `parameters_range` (priors)
and accounting for uncertainty.





- Takes a pre-trained emulator (surrogate model)
- Uses sensitivity analysis results to identify the most important parameters
- Accepts observations (real data) to calibrate against
- Optionally incorporates NROY (Not Ruled Out Yet) samples from prior history matching
- Sets up parameter bounds for calibration

In [ ]:
from autoemulate.mcmc import MCMCCalibrator
# Define your observations (what you want to match)
# Define observed data with means and variances


# Run calibration
calibrator = MCMCCalibrator(
    emulator=gp_final,
    sensitivity_results=si,
    observations=observations,
    parameter_bounds=parameters_range,
    nroy_samples=nroy_samples,
    nroy_indices=nroy_indices,
    all_samples=all_samples,
    top_n_params=3  # Calibrate top 5 most sensitive parameters
)

results = calibrator.run_mcmc(num_samples=100, warmup_steps=10)
# Get calibrated parameter values
calibrated_params = calibrator.get_calibrated_parameters()
calibrated_params


In [ ]:
from autoemulate.mcmc_dashboard import MCMCVisualizationDashboard
dashboard = MCMCVisualizationDashboard(calibrator)
dashboard.display()

### Footnote: Testing the dashboard

Sometimes it is hard to know, if the results we are seeing is because the code is not working, or our simulation results are more interesting than we expected. Here is a little test dataset which tests the dashboard, so that you can see how the plots are supposed to look liek and what they shouldf show

In [ ]:
# Create a test sample with KNOWN NROY regions
test_samples = np.array([[x, y] for x in np.linspace(0,1,100)
                               for y in np.linspace(0,1,100)])
test_scores = (abs(test_samples[:, 0]-0.5)+abs(test_samples[:, 1]-0.5)).reshape(-1, 1)

# Should show a clear diagonal pattern
test_dash = HistoryMatchingDashboard(
    samples=test_samples,
    impl_scores=test_scores,
    param_names=["p1", "p2"],
    output_names=["out1"],
    threshold=0.7  # ~50% of points should be NROY
)
#test_dash.display()